In [1]:
# del session.logger
# del session

import multiprocessing
from train.training_session_v1 import TrainingSessionV1
from config import TrainingConfigV1
from config import SimpleConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=80,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=0.0,  # Float
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Conformer encoder variant
    rnn_type="conformer",
    depthwise_conv_kernel_size=31,
    use_group_norm=True,
    convolution_first=False,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV1(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Ada lora settings
    # Around 100k total batches an epoch for gwilliams
    adalora_init_r=12,
    adalora_target_r=4,
    adalora_tinit=800,  # 5% total steps
    adalora_tfinal=12800,  # 50-80% total steps
    adalora_deltaT=400,  # 1-5% total steps
    adalora_lora_alpha=32,
    adalora_lora_dropout=0.1,
    adalora_total_step=16000,
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Audio
    audio_model="openai/whisper-tiny.en",
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-3,
    epochs=40,
    batch_size=128,
    random_test_size=10,
    seed=42,
    mel_alignment_objectives={
        "clip_loss": 0.8,
        "mse_loss": 0.2,
        "commitment_loss": 0.0,
        "cosine_similarity": 0.0,
    },
    latent_alignment_objectives={
        "cosine_similarity": 0.4,
        "mse_loss": 0.4,
        "clip_loss": 0.2,
    },
    latent_alignment_layers=[-1],
)

config.brain_encoder_config.mel_normalization = False
config.learning_rate = 6e-4
config.batch_size = 128

session = TrainingSessionV1(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="/home/ubuntu/storage-texas/data",
    save_path="saves/phase2/objectives/CLIP_MSE_BATCH_LEVEL_128_6e-4",
    clear_cache=False,
    cache_name="cache",
    download_studies=False,
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() // 2 - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

# try:
#     session.pre_process_all_recordings(
#         buffer_size=30, num_workers=multiprocessing.cpu_count() - 2, max_cache_size=400
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

SimpleConv initialized with 9295984 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 6, hidden_dim: 384, params 8858112
Found 24 target modules for AdaLora: ['k_proj', 'q_proj', 'v_proj', 'out_proj', 'fc1', 'fc2']
openai/whisper-tiny.en loaded with 8540472 frozen params (4 layers and 384 dim).
AdaLora has 332064 trainable params.


2025-02-05 00:24:47,912	INFO worker.py:1841 -- Started a local Ray instance.
Training Epoch 1:  92%|█████████▏| 124/135 [10:56<00:48,  4.44s/it]

Starting rank reallocation at step 800.


Training Epoch 1: 100%|██████████| 135/135 [11:37<00:00,  5.16s/it]


Testing completed in 0.95m.
Epoch 1 completed in 12.59m. 0.09m per recording.
New highest average test accuracy: 0.0207, lowest FinLayer total loss: 1.2606 at epoch 1.


...

Testing completed in 0.97m.
Epoch 38 completed in 12.78m. 0.09m per recording.


Training Epoch 39: 100%|██████████| 135/135 [11:35<00:00,  5.15s/it]


Testing completed in 0.95m.
Epoch 39 completed in 12.54m. 0.09m per recording.


Training Epoch 40: 100%|██████████| 135/135 [11:28<00:00,  5.10s/it]


Testing completed in 0.96m.
Epoch 40 completed in 12.43m. 0.09m per recording.
Training completed.
unseen_subject: Acc: 0.0230, Top 5: 0.1035, Top 10: 0.1950
Loss: 4.5574, Mel Loss: 3.4791, Clip Loss: 4.2056, MSE Loss: 0.5730, CosSim Loss: 0.0000, Commit Loss: 0.0000
FinLayer Clip Loss: 3.7968, FinLayer MSE Loss: 0.5782
FinLayer CosSim Loss: 0.2192, FinLayer Total Loss: 1.0783
unseen_task: Acc: 0.0185, Top 5: 0.0903, Top 10: 0.1671
Loss: 4.7222, Mel Loss: 3.6129, Clip Loss: 4.3707, MSE Loss: 0.5819, CosSim Loss: 0.0000, Commit Loss: 0.0000
FinLayer Clip Loss: 3.9333, FinLayer MSE Loss: 0.5854
FinLayer CosSim Loss: 0.2211, FinLayer Total Loss: 1.1093
unseen_both: Acc: 0.0225, Top 5: 0.1050, Top 10: 0.2013
Loss: 4.5331, Mel Loss: 3.4576, Clip Loss: 4.1793, MSE Loss: 0.5709, CosSim Loss: 0.0000, Commit Loss: 0.0000
FinLayer Clip Loss: 3.7832, FinLayer MSE Loss: 0.5781
FinLayer CosSim Loss: 0.2192, FinLayer Total Loss: 1.0755


In [1]:
# del session.logger
# del session

import multiprocessing
from train.training_session_v1 import TrainingSessionV1
from config import TrainingConfigV1
from config import SimpleConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=80,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=0.0,  # Float
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Conformer encoder variant
    rnn_type="conformer",
    depthwise_conv_kernel_size=31,
    use_group_norm=True,
    convolution_first=False,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV1(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Ada lora settings
    # Around 100k total batches an epoch for gwilliams
    adalora_init_r=12,
    adalora_target_r=4,
    adalora_tinit=800 // 2,  # 5% total steps
    adalora_tfinal=16000,  # 50-80% total steps
    adalora_deltaT=400 // 2,  # 1-5% total steps
    adalora_lora_alpha=32,
    adalora_lora_dropout=0.1,
    adalora_total_step=20000,
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Audio
    audio_model="openai/whisper-tiny.en",
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-3,
    epochs=40,
    batch_size=128,
    random_test_size=10,
    seed=42,
    mel_alignment_objectives={
        "clip_loss": 0.8,
        "mse_loss": 0.2,
        "commitment_loss": 0.0,
        "cosine_similarity": 0.0,
    },
    latent_alignment_objectives={
        "cosine_similarity": 0.4,
        "mse_loss": 0.4,
        "clip_loss": 0.2,
    },
    latent_alignment_layers=[-1],
)

config.brain_encoder_config.mel_normalization = False
config.learning_rate = 3e-4
config.batch_size = 256

session = TrainingSessionV1(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="/home/ubuntu/storage-texas/data",
    save_path="saves/phase2/objectives/CLIP_MSE_BATCH_LEVEL_256_3e-4",
    clear_cache=False,
    cache_name="cache",
    download_studies=False,
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

# try:
#     session.pre_process_all_recordings(
#         buffer_size=30, num_workers=multiprocessing.cpu_count() - 2, max_cache_size=400
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

Loading Gwilliams2023 with batch type audio
Data partitioned on studies ['gwilliams2023'].
Train: 135, Unseen Task: 12, Unseen Subject: 45, Unseen Both: 4.

SimpleConv initialized with 9295984 parameters, cond: ['study', 'subject']
Merger False, merger channels 0
ConvBlocks: 6, hidden_dim: 384, params 8858112
Found 24 target modules for AdaLora: ['k_proj', 'q_proj', 'v_proj', 'out_proj', 'fc1', 'fc2']
openai/whisper-tiny.en loaded with 8540472 frozen params (4 layers and 384 dim).
AdaLora has 332064 trainable params.


2025-02-05 19:38:49,410	INFO worker.py:1841 -- Started a local Ray instance.
Training Epoch 1:   0%|          | 0/135 [00:00<?, ?it/s]

Exited


Training Epoch 1:   0%|          | 0/135 [00:13<?, ?it/s]


In [ ]:
# del session.logger
# del session

import multiprocessing
from train.training_session_v1 import TrainingSessionV1
from config import TrainingConfigV1
from config import SimpleConvConfig

data_partition = {
    "gwilliams2023": {
        "testing_subjects": [19, 20, 21],
        "testing_tasks": [0],
    },
    # "armeini2022": {
    #     "testing_subjects": [],
    #     "testing_tasks": [8, 9],
    # },
}

model_config = SimpleConvConfig(
    # Str to list of possible conditions
    mel_normalization=False,
    conditions={
        "study": [],
        "subject": [],
    },
    # Channels
    in_channels=208,
    out_channels=80,
    hidden_dim=384,
    dropout=0.2,
    initial_batch_norm=True,
    # Sensor layout settings
    layout_dim=2,
    layout_proj=True,
    layout_scaling="minmax",
    # Merger with spatial attn
    merger=False,
    merger_emb_type=None,
    merger_emb_dim=0,
    merger_channels=0,
    merger_dropout=0.0,  # Float
    merger_conditional=None,
    # Inital
    initial_linear=384,
    initial_depth=1,
    # Conditional layers
    conditional_layers=False,
    conditional_layers_dim=None,  # input or hidden_dim
    # Conv layer overall structure
    depth=6,
    kernel_size=3,
    growth=1.0,
    dilation_growth=2,
    dilation_period=5,
    glu=1,
    conv_dropout=0.2,
    dropout_input=0.2,
    batch_norm=True,
    half=True,
    cnn_pos_encoding=False,
    # Quantizer
    quantizer=False,
    num_codebooks=0,
    codebook_size=0,
    quantizer_commitment=0,
    quantizer_temp_init=0,
    quantizer_temp_min=0,
    quantizer_temp_decay=0,
    # Transformers Encoders
    transformer_input=None,
    transformer_encoder_emb=None,
    transformer_encoder_layers=0,
    transformer_encoder_heads=0,
    # Conformer encoder variant
    rnn_type="conformer",
    depthwise_conv_kernel_size=31,
    use_group_norm=True,
    convolution_first=False,
    # Transformer Decoders
    transformer_decoder_emb=None,
    transformer_decoder_layers=0,
    transformer_decoder_heads=0,
    transformer_decoder_dim=0,
)

config = TrainingConfigV1(
    brain_encoder_config=model_config,
    data_partition=data_partition,
    # Ada lora settings
    # Around 100k total batches an epoch for gwilliams
    adalora_init_r=12,
    adalora_target_r=4,
    adalora_tinit=800 // 2,  # 5% total steps
    adalora_tfinal=16000,  # 50-80% total steps
    adalora_deltaT=400 // 2,  # 1-5% total steps
    adalora_lora_alpha=32,
    adalora_lora_dropout=0.1,
    adalora_total_step=20000,
    # Pre-processing parameters
    # Brain
    new_freq=200,
    frequency_bands={"all": (0.5, 80)},
    max_random_shift=1.0,
    window_size=4,
    window_stride=1,
    brain_clipping=None,
    baseline_window=0.5,
    notch_filter=True,
    scaling="both",
    delay=0.15,
    # Audio
    audio_model="openai/whisper-tiny.en",
    # Hyperparameters
    learning_rate=3e-4,
    weight_decay=1e-3,
    epochs=40,
    batch_size=128,
    random_test_size=10,
    seed=42,
    mel_alignment_objectives={
        "clip_loss": 0.8,
        "mse_loss": 0.2,
        "commitment_loss": 0.0,
        "cosine_similarity": 0.0,
    },
    latent_alignment_objectives={
        "cosine_similarity": 0.4,
        "mse_loss": 0.4,
        "clip_loss": 0.2,
    },
    latent_alignment_layers=[-1],
)

config.brain_encoder_config.mel_normalization = True
config.learning_rate = 3e-4
config.batch_size = 256

session = TrainingSessionV1(
    config=config,
    studies={study: "audio" for study in data_partition.keys()},
    data_path="/home/ubuntu/storage-texas/data",
    save_path="saves/phase2/objectives/CLIP_MSE_BATCH_LEVEL_256_3e-4_TANH_NORM",
    clear_cache=False,
    cache_name="cache",
    download_studies=False,
)

try:
    session.train(
        device="cuda",
        buffer_size=30,
        num_workers=(multiprocessing.cpu_count() - 2),
        max_cache_size=400,
        current_epoch=0,
    )
except KeyboardInterrupt as e:
    print("Exited")

# try:
#     session.pre_process_all_recordings(
#         buffer_size=30, num_workers=multiprocessing.cpu_count() - 2, max_cache_size=400
#     )
# except KeyboardInterrupt as e:
#     print("Exited")

In [ ]:
# for name, param in session.model.encoder.named_parameters():
#     if "lora_A" in name or "lora_B" in name or "lora_E" in name:
#         print(name)

In [ ]:
# import time

# dataloader = session.get_dataloader(buffer_size=30, num_workers=24, max_cache_size=400)
# dataloader.start_fetching(session.dataset["train"], cache=True)

# # Process batches as they become available
# try:
#     batches, recs, start_time = 0, 0, time.time()
#     print(f"Total recordings: {len(session.dataset['train'])}")

#     while True:
#         batch = dataloader.get_recording()

#         if batch is None:
#             break

#         brain = batch.brain_segments["all"]
#         batches += brain.shape[0]
#         recs += 1

#         if recs % 10 == 0:
#             print(f'Processed {recs} recordings of {len(session.dataset["train"])}')

#     print(
#         f"Batch {batches} ({recs} recordings) processed in {time.time() - start_time:.2f}s"
#     )
#     print(
#         f"Average processing time per recording: {(time.time() - start_time) / recs:.2f}s"
#     )
#     print(
#         f"Average processing time per batch: {(time.time() - start_time) / batches:.2f}s"
#     )

# except KeyboardInterrupt:
#     print("Interrupted")
#     dataloader.stop()
# except Exception as e:
#     print("Error", e)
#     dataloader.stop()